# load package and model

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path

from config_ai.models.mlm import TransformerMLMModel
from config_ai.evaluate import eval_mlm
from config_ai.models.mlm.common import *
from config_ai.data_utils import *
from config_ai.backend import set_tf_config
from config_ai.models import load_model
from config_ai.experiments import get_model_config
from config_ai.utils import read_config
from snippets import *

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
set_tf_config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['/nfs/pony/chenhao/workspace/ConfigAI/notebooks/mlm',
 '/nfs/pony/chenhao/workspace/python-snippets',
 '/nfs/pony/chenhao/workspace/ConfigAI',
 '/nfs/pony/chenhao/workspace/bert4keras',
 '/nfs/pony/chenhao/workspace/ConfigAI/notebooks/mlm',
 '/nfs/pony/chenhao/envs/config_ai/lib/python38.zip',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/lib-dynload',
 '',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages/IPython/extensions',
 '/mnt/raid4/home/chenhao/.ipython']

2021-11-23 15:45:52 [INFO][backend.py:57]:setting tensorflow config...
2021-11-23 15:45:52.711039: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-23 15:45:52 [INFO][backend.py:61]:current devices:
2021-11-23 15:45:52.724519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
coreClock: 1.531GHz coreCount: 28 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 447.48GiB/s
2021-11-23 15:45:52.724920: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-23 15:45:52.726669: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-23 15:45:52.728091: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-1

In [3]:
config_path = "/nfs/pony/chenhao/workspace/ConfigAI/examples/mlm/poetry_tranformer_mlm.ini"

config = read_config(config_path)
config
model_config = get_model_config(config)
# model_config

model = TransformerMLMModel(config=model_config)


2021-11-23 15:45:58 [INFO][utils.py:60]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/examples/mlm/poetry_tranformer_mlm.ini
2021-11-23 15:45:58 [INFO][utils.py:71]:loading base config...
2021-11-23 15:45:58 [INFO][utils.py:60]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/examples/base_config.ini


{'common_config': {'experiment_dir': '/nfs/pony/chenhao/experiment',
  'project_name': 'config_ai_example',
  'is_train': True,
  'is_test': True,
  'is_save': True,
  'save_args': {'format': 'h5', 'tf_serving_version': 1},
  'eval_phase_list': ['train', 'dev'],
  'output_phase_list': ['train', 'dev', 'test'],
  'is_overwrite_experiment': True,
  'default_random_seed': 10,
  'base_config': '/nfs/pony/chenhao/workspace/ConfigAI/examples/base_config.ini',
  'model_cls': 'TransformerMLMModel',
  'model_name': 'transformer_mlm_example'},
 'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'nn_model_config': {'pretrained_model_tag': 'bert',
  'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'},
 'compile_config': {'optimizer_name': 'adam',
  'optimizer_args': {'learning_rate': 3e-05}},
 'train_config': {'epochs': 5, 'batch_size': 32, 'overwrite_cache': 

2021-11-23 15:45:58 [INFO][core.py:47]:init model with config:
2021-11-23 15:45:58 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-23 15:45:58 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs


# process data

In [4]:
train_data_path = config['data_config']['train_data_path']
train_data = model.jload_lines(train_data_path)
f"{train_data_path}: {len(train_data)} items"


eval_data_path = config['data_config']['eval_data_path']
eval_data = model.jload_lines(eval_data_path)
f"{eval_data_path}: {len(eval_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.jload_lines(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'/nfs/pony/chenhao/workspace/ConfigAI/data/poetry/unlabeled.jsonl: 3 items'

'/nfs/pony/chenhao/workspace/ConfigAI/data/poetry/test.jsonl: 3 items'

'/nfs/pony/chenhao/workspace/ConfigAI/data/poetry/test.jsonl: 3 items'

MLMExample(text='白日依山尽，黄河入海流。欲穷千里目，更上一层楼。', masked_tokens=None)

In [5]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [6]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '白日依山尽，黄河入海流。欲穷千里目，更上一层楼。',
 'text': '白日依山尽，黄河入海流。欲穷千里目，更上一层楼。',
 'extra_text': None,
 'token_ids': [101,
  4635,
  3189,
  898,
  2255,
  2226,
  8024,
  7942,
  3777,
  1057,
  3862,
  3837,
  511,
  3617,
  4956,
  1283,
  7027,
  4680,
  8024,
  3291,
  677,
  671,
  2231,
  3517,
  511,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '白',
  '日',
  '依',
  '山',
  '尽',
  '，',
  '黄',
  '河',
  '入',
  '海',
  '流',
  '。',
  '欲',
  '穷',
  '千',
  '里',
  '目',
  '，',
  '更',
  '上',
  '一',
  '层',
  '楼',
  '。',
  '[SEP]'],
 'masks': []}

In [7]:
records = data_manager.get_records(mode="train")
next(records)



{'idx': 0,
 'full_text': '白日依山尽，黄河入海流。欲穷千里目，更上一层楼。',
 'text': '白日依山尽，黄河入海流。欲穷千里目，更上一层楼。',
 'extra_text': None,
 'token_ids': [101,
  103,
  3189,
  898,
  2255,
  2226,
  8024,
  7942,
  3777,
  1057,
  3862,
  3837,
  511,
  3617,
  4956,
  103,
  7027,
  4680,
  8024,
  3291,
  677,
  671,
  2231,
  103,
  511,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '[MASK]',
  '日',
  '依',
  '山',
  '尽',
  '，',
  '黄',
  '河',
  '入',
  '海',
  '流',
  '。',
  '欲',
  '穷',
  '[MASK]',
  '里',
  '目',
  '，',
  '更',
  '上',
  '一',
  '层',
  '[MASK]',
  '。',
  '[SEP]'],
 'masks': [],
 'token_output': [0,
  4635,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1283,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3517,
  0,
  0],
 'masked_tokens': [(1, '白'), (15, '千'), (23, '楼')]}

# build&compile model

In [8]:
nn_model_args = dict(**config["nn_model_config"])
nn_model_args

model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'}

2021-11-23 15:46:04.253620: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-23 15:46:04.264634: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3597740000 Hz
2021-11-23 15:46:04.265293: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dc98410530 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-23 15:46:04.265324: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-23 15:46:04.434844: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dc984aaa40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-23 15:46:04.434873: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2021-11-23 15:46:04.435894: I tensorflow

In [9]:
compile_args = dict(**config["compile_config"])
# compile_args.update(rdrop_alpha=4)
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-11-23 15:46:09 [INFO][transformer_mlm.py:59]:compiling model...
2021-11-23 15:46:09 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-11-23 15:46:09 [INFO][losses.py:101]:build loss layer with loss function:<function sparse_categorical_crossentropy at 0x7f6ba7527dc0>
2021-11-23 15:46:10 [INFO][transformer_mlm.py:78]:training model's summary:
2021-11-23 15:46:10 [INFO][layer_utils.py:192]:Model: "train_model"
2021-11-23 15:46:10 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-23 15:46:10 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-11-23 15:46:10 [INFO][layer_utils.py:195]:==================================================================================================
2021-11-23 15:46:10 [INFO][layer_utils.py:190]:token_ids (InputLayer)          [(None, None)]       0                           

In [10]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


[<tf.Tensor 'token_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'segment_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'token_output:0' shape=(None, None) dtype=int32>]

[<tf.Tensor 'model/Identity:0' shape=(None, None, 21128) dtype=float32>]

[<tf.Tensor 'loss_layer/Identity:0' shape=() dtype=float32>]

In [11]:
model.get_dataset_info("train")
model.get_dataset_info("test")

({'token_ids': 'int32', 'segment_ids': 'int32', 'token_output': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'token_output': (None,)})

({'token_ids': 'int32', 'segment_ids': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,)})

In [12]:
dataset = data_manager.get_train_dataset(batch_size=1)

item = next(dataset.as_numpy_iterator())
item


{'token_ids': array([[ 101,  103, 3189,  898, 2255, 2226, 8024, 7942, 3777, 1057, 3862,
         3837,  511, 3617, 4956, 1283, 7027, 4680,  103, 3291,  677,  671,
         2231, 3517,  511,  102]], dtype=int32),
 'segment_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], dtype=int32),
 'token_output': array([[   0, 4635,    0,    0,    0,    0,    0,    0,    0,    0, 3862,
            0,    0,    0,    0,    0,    0,    0, 8024,    0,    0,    0,
            0,    0,    0,    0]], dtype=int32)}

# train_model

In [13]:
train_args = dict(**config["train_config"])
train_args.update(steps_per_epoch=100, batch_size=8, verbose=1)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 5,
 'batch_size': 8,
 'overwrite_cache': False,
 'steps_per_epoch': 100,
 'verbose': 1}

100%|██████████| 3/3 [00:00<00:00, 3552.49it/s]
2021-11-23 15:46:15 [INFO][tf_core.py:173]:train on 3 tensors


Epoch 1/5


/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2021-11-23 15:46:20.277583: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


100/100 [==============================] - 11s 110ms/step - loss: 1.2567 - accuracy: 0.7906
Epoch 2/5
100/100 [==============================] - 11s 106ms/step - loss: 0.0553 - accuracy: 0.9868
Epoch 3/5
100/100 [==============================] - 11s 105ms/step - loss: 0.0179 - accuracy: 0.9966
Epoch 4/5
100/100 [==============================] - 11s 107ms/step - loss: 0.0230 - accuracy: 0.9940
Epoch 5/5
100/100 [==============================] - 11s 108ms/step - loss: 0.0084 - accuracy: 0.9992


2021-11-23 15:47:14 [INFO][tf_core.py:189]:training finished


# predict&evaluate

In [14]:
to_pred = test_data[:4]
to_pred

pred = model.predict(to_pred, show_detail=True)
pred[:4]


[MLMExample(text='白日依山尽，黄[MASK]入海流。欲穷千里目，更上一层楼。', masked_tokens=['河']),
 MLMExample(text='床前明月光，疑是[MASK][MASK]霜。举头望明月，低头思故[MASK]。', masked_tokens=['地', '上', '乡']),
 MLMExample(text='日照香炉生紫烟，遥望瀑布挂前川。飞流直下三千尺[MASK][MASK]是银河落九天。', masked_tokens=['，', '疑'])]

100%|██████████| 3/3 [00:00<00:00, 2266.78it/s]
2021-11-23 15:47:17 [INFO][tf_core.py:204]:predicting with tf model...
1it [00:00,  7.24it/s]
2021-11-23 15:47:17 [INFO][decorators.py:28]:function:_model_predict cost:0.140 seconds
2021-11-23 15:47:17 [INFO][decorators.py:28]:function:_post_predict cost:0.008 seconds


[['河'], ['地', '上', '乡'], ['，', '疑']]

In [16]:
output_data = get_mlm_output(to_pred, pred)
output_data[:4]

[{'text': '白日依山尽，黄[MASK]入海流。欲穷千里目，更上一层楼。',
  'masked_tokens': ['河'],
  'predict': ['河'],
  'accuracy': 1.0},
 {'text': '床前明月光，疑是[MASK][MASK]霜。举头望明月，低头思故[MASK]。',
  'masked_tokens': ['地', '上', '乡'],
  'predict': ['地', '上', '乡'],
  'accuracy': 1.0},
 {'text': '日照香炉生紫烟，遥望瀑布挂前川。飞流直下三千尺[MASK][MASK]是银河落九天。',
  'masked_tokens': ['，', '疑'],
  'predict': ['，', '疑'],
  'accuracy': 1.0}]

In [19]:
true_labels = [e.masked_tokens for e in to_pred]
eval_rs = eval_mlm(true_labels, pred)
eval_rs

{'item_num': 3, 'token_num': 6, 'accurate_token_num': 6, 'accuracy': 1.0}

# save&load

In [20]:
model_path = os.path.join(config["common_config"]["experiment_dir"],
                          config["common_config"]["project_name"],
                          config["common_config"]["model_name"],"model")

save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'/nfs/pony/chenhao/experiment/config_ai_example/transformer_mlm_example/model'

2021-11-23 15:48:29 [INFO][core.py:79]:saving model to /nfs/pony/chenhao/experiment/config_ai_example/transformer_mlm_example/model
2021-11-23 15:48:29 [INFO][tf_core.py:55]:saving keras model to path:/nfs/pony/chenhao/experiment/config_ai_example/transformer_mlm_example/model/nn_model/nn_model.h5
2021-11-23 15:48:31 [INFO][core.py:144]:save model done


In [ ]:
loaded_model = load_model(path=model_path)
loaded_model

2021-11-23 15:48:34 [INFO][core.py:90]:loading model from path:/nfs/pony/chenhao/experiment/config_ai_example/transformer_mlm_example/model
2021-11-23 15:48:34 [INFO][core.py:47]:init model with config:
2021-11-23 15:48:34 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-23 15:48:34 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs
2021-11-23 15:48:34 [INFO][tf_core.py:157]:loading keras model from path:/nfs/pony/chenhao/experiment/config_ai_example/transformer_mlm_example/model/nn_model/nn_model.h5 with format:h5


In [20]:
loaded_pred = loaded_model.predict(data=to_pred)
loaded_pred

100%|██████████| 3/3 [00:00<00:00, 11285.12it/s]
2021-11-22 15:05:23 [INFO][tf_core.py:204]:predicting with tf model...
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_model_predict cost:0.036 seconds
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_post_predict cost:0.001 seconds


[Label(name='正向', prob=0.9998229146003723),
 Label(name='负向', prob=0.9997743964195251),
 Label(name='中性', prob=0.9997645020484924)]